In [ ]:
import asyncio
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import math
np.set_printoptions(threshold=np.inf, linewidth=np.inf) #inf = infinity
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
import string
import time
from datetime import datetime
import logging

from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.aws import AwsDevice, AwsQuantumTask
from braket.devices import LocalSimulator

device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Harmony") 

# Identity

In [ ]:
id00 = Circuit().rz(0,4*pi).rz(1,4*pi)
id01 = Circuit().rz(0,4*pi).x(1)
id10 = Circuit().x(0).rz(1,4*pi)
id11 = Circuit().x(0).x(1)

In [54]:
taski00 = device.run(id00, shots=1000)
taski01 = device.run(id01, shots=1000)
taski10 = device.run(id10, shots=1000)
taski11 = device.run(id11, shots=1000)

task_i00 = taski00.id
task_i01 = taski01.id
task_i10 = taski10.id
task_i11 = taski11.id

print("00", task_i00)
print("01", task_i01)
print("10", task_i10)
print("11", task_i11)

00 arn:aws:braket:us-east-1:291250255500:quantum-task/2ea5cb38-e462-41e1-a926-0dd1b0ded8bb
01 arn:aws:braket:us-east-1:291250255500:quantum-task/9c8405e3-fb52-497d-b23a-de83e7697c02
10 arn:aws:braket:us-east-1:291250255500:quantum-task/f0f5a867-fc1b-4b3d-a012-f3f1a1ca6421
11 arn:aws:braket:us-east-1:291250255500:quantum-task/10721166-688a-40c6-b991-07a12722bf61


In [59]:
task_load_i00 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/2ea5cb38-e462-41e1-a926-0dd1b0ded8bb')
task_load_i01 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/9c8405e3-fb52-497d-b23a-de83e7697c02')
task_load_i10 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/f0f5a867-fc1b-4b3d-a012-f3f1a1ca6421')
task_load_i11 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/10721166-688a-40c6-b991-07a12722bf61')

print("00: ", task_load_i00.result().measurement_counts)
print("01: ", task_load_i01.result().measurement_counts)
print("10: ", task_load_i10.result().measurement_counts)
print("11: ", task_load_i11.result().measurement_counts)

00:  Counter({'00': 994, '01': 4, '10': 2})
01:  Counter({'01': 984, '00': 15, '11': 1})
10:  Counter({'10': 972, '00': 24, '11': 4})
11:  Counter({'11': 976, '01': 17, '10': 7})


# SWAP

In [ ]:
sw00 = Circuit().swap(0,1)
sw01 = Circuit().x(1).swap(0,1)
sw10 = Circuit().x(0).swap(0,1)
sw11 = Circuit().x(0).x(1).swap(0,1)

In [57]:
tasks00 = device.run(sw00, shots=1000)
tasks01 = device.run(sw01, shots=1000)
tasks10 = device.run(sw10, shots=1000)
tasks11 = device.run(sw11, shots=1000)

task_s00 = tasks00.id
task_s01 = tasks01.id
task_s10 = tasks10.id
task_s11 = tasks11.id

print("00", task_s00)
print("01", task_s01)
print("10", task_s10)
print("11", task_s11)

00 arn:aws:braket:us-east-1:291250255500:quantum-task/678eef21-8e4a-47d5-96b1-a89d5fbc1c5d
01 arn:aws:braket:us-east-1:291250255500:quantum-task/0550e4a9-c8eb-451b-9248-38ea70ba5f42
10 arn:aws:braket:us-east-1:291250255500:quantum-task/41c93c2d-68cb-4670-89dc-77828586be29
11 arn:aws:braket:us-east-1:291250255500:quantum-task/68c4b43b-4fbd-43bc-b51e-f22e184bd63b


In [60]:
task_load_s00 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/678eef21-8e4a-47d5-96b1-a89d5fbc1c5d')
task_load_s01 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/0550e4a9-c8eb-451b-9248-38ea70ba5f42')
task_load_s10 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/41c93c2d-68cb-4670-89dc-77828586be29')
task_load_s11 = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/68c4b43b-4fbd-43bc-b51e-f22e184bd63b')

print("00: ", task_load_s00.result().measurement_counts)
print("01: ", task_load_s01.result().measurement_counts)
print("10: ", task_load_s10.result().measurement_counts)
print("11: ", task_load_s11.result().measurement_counts)

00:  Counter({'00': 948, '01': 22, '10': 16, '11': 14})
01:  Counter({'10': 941, '00': 29, '11': 21, '01': 9})
10:  Counter({'01': 942, '00': 30, '10': 15, '11': 13})
11:  Counter({'11': 935, '01': 30, '10': 20, '00': 15})


# SWAP_eigenvector

In [ ]:
sw_ev1 = Circuit()
sw_ev1.swap(0,1)

sw_ev2 = Circuit().x(1).h(0).cnot(0,1)
sw_ev2.swap(0,1)

sw_ev3 = Circuit().x(0).x(1).h(0).cnot(0,1)
sw_ev3.swap(0,1)

sw_ev4 = Circuit().x(0).x(1)
sw_ev4.swap(0,1)

In [14]:
tasks00e = device.run(sw_ev1, shots=1000)
tasks01e = device.run(sw_ev2, shots=1000)
tasks10e = device.run(sw_ev3, shots=1000)
tasks11e = device.run(sw_ev4, shots=1000)

task_s00e = tasks00e.id
task_s01e = tasks01e.id
task_s10e = tasks10e.id
task_s11e = tasks11e.id

print("ev1", task_s00e)
print("ev2", task_s01e)
print("ev3", task_s10e)
print("ev4", task_s11e)

ev1 arn:aws:braket:us-east-1:291250255500:quantum-task/4d0e2564-7c84-43f5-8373-b8a02e990ca1
ev2 arn:aws:braket:us-east-1:291250255500:quantum-task/5f6ad18a-abc1-4aaf-af33-e23e648febef
ev3 arn:aws:braket:us-east-1:291250255500:quantum-task/3fbd9338-1131-4f62-9723-9ab4a93441c0
ev4 arn:aws:braket:us-east-1:291250255500:quantum-task/cd126379-3811-4152-a9dc-6d03dd2ad64f


In [16]:
task_load_s00e = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/4d0e2564-7c84-43f5-8373-b8a02e990ca1')
task_load_s01e = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/5f6ad18a-abc1-4aaf-af33-e23e648febef')
task_load_s10e = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/3fbd9338-1131-4f62-9723-9ab4a93441c0')
task_load_s11e = AwsQuantumTask(arn = 'arn:aws:braket:us-east-1:291250255500:quantum-task/cd126379-3811-4152-a9dc-6d03dd2ad64f')

print("00: ", task_load_s00e.result().measurement_counts)
print("01: ", task_load_s01e.result().measurement_counts)
print("10: ", task_load_s10e.result().measurement_counts)
print("11: ", task_load_s11e.result().measurement_counts)

00:  Counter({'00': 955, '10': 18, '01': 17, '11': 10})
01:  Counter({'01': 518, '10': 453, '00': 23, '11': 6})
10:  Counter({'10': 493, '01': 474, '00': 23, '11': 10})
11:  Counter({'11': 972, '01': 12, '10': 11, '00': 5})
